# Lima Buendía

### Extracción de datos y creación de datasets limpios

#### Obejtivo
La lectura de datos se realiza desde un bucket de s3-
 
De manera general, se tienen cuatro fuentes de datos de redes sociales -tres de redes sociales y una de encabezados de noticias.
+ Meta Cloud
+ Twitter
+ FacebookPages
+ FacebookGroup

Y dos más de datos oficiales: 
+ Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública 
+ INEGI

In [ ]:
#Carga de librerías
import boto3
import pandas as pd
import io
from io import StringIO
import numpy as np
import unicodedata
import re


from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

In [ ]:
# Cargamos las librerías de dotenv
import dotenv
from dotenv import load_dotenv
import os
from dotenv import dotenv_values

In [ ]:
# Obtener el ambiente
load_dotenv()

In [ ]:
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [ ]:
# Preparacion de conexiones 
client = boto3.client('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = 'us-east-1'
)
    
# Creating the high level object oriented interface
resource = boto3.resource('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = 'us-east-1'
)

session = boto3.Session(aws_access_key_id=aws_access_key_id,
                        aws_secret_access_key=aws_secret_access_key
)

In [ ]:
# parámetros
s3_res = session.resource('s3')
csv_buffer = StringIO()
bucket_name = 'lima-buendia'

# Llaves
#metacloud = 
m1 = 'raw/Mediacloud_data_1ene2018a31may2022.csv'
m2 = 'raw/Mediacloud_data_1jul2020a31jul2020.csv'
m3 = 'raw/Mediacloud_data_1oct2020a31dic2020.csv'
m4 = 'raw/Mediacloud_conteo_1oct2021a30nov2021.csv'
m5 = 'raw/FBgroups Feminicidios 1ene2017a31may2022.csv'
m6 = 'raw/FBpages Feminicidios 1ene2017a31dic2019.csv'
m7 = 'raw/FBpages Feminicidios 1ene2020a31may2022.csv'
m8 = 'raw/conjunto_de_datos_defunciones_generales_2017.CSV'
m9 = 'raw/conjunto_de_datos_defunciones_registradas_2018.csv'
m10 = 'raw/conjunto_de_datos_defunciones_registradas_2019.CSV'
m11 = 'raw/conjunto_de_datos_defunciones_registradas_2020.csv'
m12 = 'raw/catalogos_2017/decatcausa.csv'
m13 = 'raw/catalogos_2018/causa_defuncion.csv'
m14 = 'raw/catalogos_2019/decatcausa.CSV'
m15 = 'raw/catalogos_2020/causa_defuncion.CSV'
m16 = 'raw/TW_positivos_negativos.csv'

twitter = 'raw/TW_data_1ene2017a31may2022.csv'
secretariado = 'raw/Estatal-V¡ctimas-2015-2022_may2022.xlsx'

In [ ]:
# Creacion de objetos
objm1 = client.get_object(Bucket = bucket_name, Key = m1)
objm2 = client.get_object(Bucket = bucket_name, Key = m2)
objm3 = client.get_object(Bucket = bucket_name, Key = m3)
objm4 = client.get_object(Bucket = bucket_name, Key = m4)
objm5 = client.get_object(Bucket = bucket_name, Key = m5)
objm6 = client.get_object(Bucket = bucket_name, Key = m6)
objm7 = client.get_object(Bucket = bucket_name, Key = m7)
objm8 = client.get_object(Bucket = bucket_name, Key = m8)
objm9 = client.get_object(Bucket = bucket_name, Key = m9)
objm10 = client.get_object(Bucket = bucket_name, Key = m10)
objm11 = client.get_object(Bucket = bucket_name, Key = m11)
objm12 = client.get_object(Bucket = bucket_name, Key = m12)
body12 = objm12['Body']
csv_string12 = body12.read().decode('latin')
objm13 = client.get_object(Bucket = bucket_name, Key = m13)
body13 = objm13['Body']
csv_string13 = body13.read().decode('latin')
objm14 = client.get_object(Bucket = bucket_name, Key = m14)
body14 = objm14['Body']
csv_string14 = body14.read().decode('latin')
objm15 = client.get_object(Bucket = bucket_name, Key = m15)
body15 = objm15['Body']
csv_string15 = body15.read().decode('latin')
objm16 = client.get_object(Bucket = bucket_name, Key = m16)


objt1 = client.get_object(Bucket = bucket_name, Key = twitter)
obje = client.get_object(Bucket = bucket_name,  Key = secretariado)

#### Lectura de datos de S3

In [ ]:
# mediacloud
df_m1 = pd.read_csv(objm1['Body'])
df_m2 = pd.read_csv(objm2['Body'])
df_m3 = pd.read_csv(objm3['Body'])
df_m4 = pd.read_csv(objm4['Body'])

#Facebook groups
df_m5 = pd.read_csv(objm5['Body'])

#Facebook pages
df_m6 = pd.read_csv(objm6['Body'])
df_m7 = pd.read_csv(objm7['Body'])

In [ ]:
#defunciones
df_m8 = pd.read_csv(objm8['Body'])
df_m9 = pd.read_csv(objm9['Body'])
df_m10 = pd.read_csv(objm10['Body'])
df_m11 = pd.read_csv(objm11['Body'])

#catalogos y bitacoras
df_m12 = pd.read_csv(StringIO(csv_string12))
df_m13 = pd.read_csv(StringIO(csv_string13))
df_m14 = pd.read_csv(StringIO(csv_string14))
df_m15 = pd.read_csv(StringIO(csv_string15))

In [ ]:
#twitter comentarios positivos y negativos
df_m16 = pd.read_csv(objm16['Body']) 
        
#twitter
df2 = pd.read_csv(objt1['Body'])

#secretariado. estadísticas estatales
estatal =pd.read_excel(io.BytesIO(obje['Body'].read()))

In [ ]:
stop_words= list(stopwords.words('spanish'))
stop_words.extend(("https","co", "si",  "tras", "años","https", "t"))

In [ ]:
#Funciones 

# Función para obtener la fecha en mes y año de Facebook Pages y Groups
def funcion_fecha_facebook(df):
    df['Post Created'] = pd.to_datetime(df['Post Created'])
    df['year'], df['month'], df['day'], df['hour'], \
    df['minute'], df['second'] = df['Post Created'].dt.year,\
    df['Post Created'].dt.month, df['Post Created'].dt.day, df['Post Created'].dt.hour,\
    df['Post Created'].dt.minute, df['Post Created'].dt.second
    df['date']= df['Post Created'].dt.date
    df['YM'] = df['Post Created'].dt.to_period('M')

#limpieza y separacion por tokens
def limpiar_tokenizar(texto):
    nuevo_texto = texto.lower()
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    regex = '[\\!\\"\\#\\{\'+}\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~\\“\\´\\‘\\”]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto) 
    nuevo_texto = nuevo_texto.strip()
    nuevo_texto = nuevo_texto.split(sep = ' ')
    nuevo_texto = [token for token in nuevo_texto if not token in stop_words]
    nuevo_texto = [token for token in nuevo_texto if len(token) > 2]
    return(nuevo_texto)

# Limpieza sin eliminar números ni emojis
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFKD', s)
      if unicodedata.category(c)!='Mn')

def limpia(s):
    s = unicode_to_ascii(s)
    s = s.upper().strip()
    regex = '[\\!\\"\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    s = re.sub(regex, ' ', s) 
    s = re.sub("\\s+", ' ', s)
    s = re.sub(r'^https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)
    s = ' '.join(cad for cad in  s.split(' ') if cad !='')
    return s.lower()

def limpia1(s):
    s = s.replace('https', '').replace('www', '').replace('youtube', '').replace(' com ', '').replace('http', '').replace('facebook', '')  
    s = re.sub("\\s+", ' ', s)
    return s

# Nos quedamos con mujeres y homicidio
def funcion(df):
    df = df[df['sexo'] == 2]
    df = df[df['presunto'] == 2]
    return df


# Obtenemos la fecha de mes y año para los datos de defunción de INEGI
def funcion_fecha_def(df):
    df = df.reset_index()
    df['anio_regis'] = df['anio_regis'].astype('str')
    df['mes_regis'] = df['mes_regis'].astype('str')
    df['time'] = df['anio_regis'] + '-' + df['mes_regis']
    df['time'] = pd.to_datetime(df['time'])
    df['YM'] = df['time'].map(lambda x: x.strftime('%Y-%m'))
    df = pd.concat([df], axis=1)
    return df

### Lectura, limpieza y transformación de datos

#### Mediacloud

In [ ]:
# Creación de dataset de mediacloud
df = pd.concat([df_m1, df_m2, df_m3])

#Transformación del tipo de fecha
df['publish_date'] = pd.to_datetime(df['publish_date'], format="%Y-%m-%d")

#Creación de componentes de fecha
df['year'], df['month'], df['day'] = df['publish_date'].dt.year,\
df['publish_date'].dt.month, df['publish_date'].dt.day


df['date']= df['publish_date'].dt.date
df['YM']=df['publish_date'].dt.to_period('M')

df['title_tokenizado'] = df['title'].apply(lambda x: limpiar_tokenizar(x))

In [ ]:
df.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/cleaned_mediacloud.csv').put(Body=csv_buffer.getvalue())

Para octubre y noviembre de 2021, no se tienen los encabezados de Mediacloud. Únicamente los conteos diarios.

In [ ]:
df_m4['date'] = pd.to_datetime(df_m4['date'], format="%Y-%m-%d")

#Creación de componentes de fecha
df_m4['year'], df_m4['month'], df_m4['day'] = df_m4['date'].dt.year,\
df_m4['date'].dt.month, df_m4['date'].dt.day
df_m4['YM']=df_m4['date'].dt.to_period('M')

In [ ]:
df_m4.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/cleaned_metacloud_oct_nov.csv').put(Body=csv_buffer.getvalue())

#### Datos estatales

In [ ]:
estatal1 = estatal[(estatal.Sexo=='Mujer') & (estatal.Año>2015)]

estatal1 = estatal1[['Año', 'Tipo de delito', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']]

estatal1_melt = pd.melt(estatal1, id_vars=['Año', 'Tipo de delito'], value_vars=[ 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'])

estatal1_melt['mes'] = np.where(estatal1_melt['variable']=='Enero', '01',
                                np.where(estatal1_melt['variable']=='Febrero', '02',
                                         np.where(estatal1_melt['variable']=='Marzo', '03',
                                                  np.where(estatal1_melt['variable']=='Abril', '04', 
                                                           np.where(estatal1_melt['variable']=='Mayo', '05',
                                                                    np.where(estatal1_melt['variable']=='Junio', '06',
                                                                             np.where(estatal1_melt['variable']=='Julio', '07',
                                                                                      np.where(estatal1_melt['variable']=='Agosto', '08',
                                                                                               np.where(estatal1_melt['variable']=='Septiembre', '09',
                                                                                                        np.where(estatal1_melt['variable']=='Octubre', '10',
                                                                                                                 np.where(estatal1_melt['variable']=='Noviembre', '11', 
                                                                                                                          '12')))))))))))




estatal1_melt['Año'] = estatal1_melt['Año'].astype('str')
estatal1_melt['mes'] = estatal1_melt['mes'].astype('str')
estatal1_melt['YM'] = estatal1_melt['Año'] + '-' + estatal1_melt['mes']

In [ ]:
d_agregados = pd.DataFrame(estatal1_melt.groupby(['YM', 'Tipo de delito']).agg({'value':lambda x: x.dropna().sum()}).rename(columns={'value':'delitos'})).reset_index()

In [ ]:
d_agregados.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/agregados_nacionales.csv').put(Body=csv_buffer.getvalue())

#### Twitter

In [ ]:
#Transformación del timestamp
df2['created_at']=pd.to_datetime(df2['created_at'])

#Creación de componentes de fecha
df2['year'], df2['month'], df2['day'], df2['hour'], \
df2['minute'], df2['second'] = df2['created_at'].dt.year,\
df2['created_at'].dt.month, df2['created_at'].dt.day, df2['created_at'].dt.hour,\
df2['created_at'].dt.minute, df2['created_at'].dt.second

df2['date']= df2['created_at'].dt.date
df2['YM']=df2['created_at'].dt.to_period('M')

df2['text_tokenizado'] = df2['text'].apply(lambda x: limpiar_tokenizar(x))

In [ ]:
df2.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/cleaned_twitter.csv').put(Body=csv_buffer.getvalue())

#### Facebook Groups & Pages

In [ ]:
funcion_fecha_facebook(df_m5)

In [ ]:
df_m7 = pd.concat([df_m6, df_m7]) 

In [ ]:
funcion_fecha_facebook(df_m7)

In [ ]:
df_m5['Message'] = df_m5['Message'].astype('str')

In [ ]:
df_m5['Clean_Message'] = df_m5['Message'].map(limpia)

In [ ]:
df_m5['Clean_Message'] = df_m5['Clean_Message'].map(limpia1)

In [ ]:
df_m7['Message'] = df_m7['Message'].astype('str')

In [ ]:
df_m7['Clean_Message'] = df_m7['Message'].map(limpia)

In [ ]:
df_m7['Clean_Message'] = df_m7['Clean_Message'].map(limpia1)

In [ ]:
df_m5.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/df_FBgroups_clean.csv').put(Body=csv_buffer.getvalue())

In [ ]:
df_m7.to_csv(csv_buffer)

In [ ]:
s3_res.Object(bucket_name, 'clean/df_FBpages_clean.csv').put(Body=csv_buffer.getvalue())

#### Defunciones INEGI

In [ ]:
df_m8 = funcion(df_m8)

In [ ]:
df_m9 = funcion(df_m9)

In [ ]:
df_m10 = funcion(df_m10)

In [ ]:
df_m11 = funcion(df_m11)

In [ ]:
# Cambiamos las claves de la causa de defunción
df_m12 = df_m12.set_index('CVE').T.to_dict('list')
df_m13 = df_m13.set_index('CVE').T.to_dict('list')
df_m14 = df_m14.set_index('cve').T.to_dict('list')
df_m15 = df_m15.set_index('cve').T.to_dict('list')

In [ ]:
df_m8['causa_def'] = df_m8['causa_def'].map(df_m12) 
df_m9['causa_def'] = df_m9['causa_def'].map(df_m13)
df_m10['causa_def'] = df_m10['causa_def'].map(df_m14)
df_m11['causa_def'] = df_m11['causa_def'].map(df_m15)

In [ ]:
df_m8 = funcion_fecha_def(df_m8)

In [ ]:
df_m9 = funcion_fecha_def(df_m9)

In [ ]:
df_m10 = funcion_fecha_def(df_m10)

In [ ]:
df_m11 = funcion_fecha_def(df_m11)

In [ ]:
df_m8.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/clean_feminicidios_2017.csv').put(Body=csv_buffer.getvalue())

In [ ]:
df_m9.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/clean_feminicidios_2018.csv').put(Body=csv_buffer.getvalue())

In [ ]:
df_m10.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/clean_feminicidios_2019.csv').put(Body=csv_buffer.getvalue())

In [ ]:
df_m11.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/clean_feminicidios_2020.csv').put(Body=csv_buffer.getvalue())

#### Tweets positivos y negativos

In [ ]:
# Obtenemos el total de tweets 
df_m16['Total_tweets'] = df_m16['tuits_negativos'] + df_m16['tuits_positivos']

In [ ]:
# Obtenemos el año y el mes
df_m16['fecha'] = pd.to_datetime(df_m16['fecha'])
df_m16['YM'] = df_m16['fecha'].map(lambda x: x.strftime('%Y-%d'))

In [ ]:
df_m16.to_csv(csv_buffer)
s3_res.Object(bucket_name, 'clean/tweets_pos_neg.csv').put(Body=csv_buffer.getvalue())